Evaluating reference data

## Loading packages

In [ ]:
import os.path as osp, os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import torch
from torchmetrics import MeanAbsolutePercentageError
from torch.nn import HuberLoss
from torch_geometric.loader import DataLoader
import os.path as osp, pickle
from gnnepcsaft.data.graphdataset import ThermoMLDataset, Ramirez, Esper
from gnnepcsaft.train.utils import  mape
from gnnepcsaft.epcsaft import utils
import polars as pl
torch.cuda.is_available()

In [ ]:
device = torch.device("cpu")

In [ ]:
pcsaft_den = utils.DenFromTensor.apply
pcsaft_vp = utils.VpFromTensor.apply

path = osp.join("gnnepcsaft/data", "thermoml")
test_loader = ThermoMLDataset(path)

In [ ]:
def test(para_data):
    data = {"inchis":[],"mden":[],"mvp":[]} 
    for gh in test_loader:
        if gh.InChI not in para_data:
            continue
        parameters = torch.as_tensor(para_data[gh.InChI][0])
        params = parameters.squeeze().to(torch.float64).numpy()
        rho = gh.rho.view(-1, 5).to(torch.float64).numpy()
        vp = gh.vp.view(-1, 5).to(torch.float64).numpy()
        mden, mvp = mape(params, rho, vp)
        data["inchis"].append(gh.InChI)
        data["mden"].append(mden)
        data["mvp"].append(mvp)
    return data

## Evaluating

In [ ]:
train_loader = Ramirez('gnnepcsaft/data/ramirez2022')
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [graph.para]
rapara = test(para_data)
rapara = pl.DataFrame(rapara)
rapara.write_csv('../reframirez.csv')

In [ ]:
train_loader = Esper("gnnepcsaft/data/esper2023")
para_data = {}
for graph in train_loader:
    para_data[graph.InChI] = [
        torch.hstack(
            [graph.para, 10 ** (graph.assoc * torch.tensor([-1.0, 1.0])), graph.munanb]
        )
    ]
rapara = test(para_data)
rapara = pl.DataFrame(rapara)
rapara.write_csv("../refesper.csv")

In [ ]:
para_data = {}
for graph in test_loader:
  try:
    para_data[graph.InChI] = [utils.parameters_gc_pcsaft(graph.smiles)]
  except BaseException as e:
    pass
rapara = test(para_data)
rapara = pl.DataFrame(rapara)
rapara.write_csv("../refgc_pcsaft.csv")